In [1]:
import polars as pl
import polars.selectors as cs
from utils import PathsData, import_data, split_data, plot_correlation
from features import FeaturesFrame

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

x_validation = import_data(PathsData.X_TEST.value) #to use in the end to make a submission
x = import_data(PathsData.X_TRAIN.value)
y = import_data(PathsData.Y_TRAIN.value)

x_train, x_test, y_train, y_test = split_data(x=x, y=y)

In [2]:
test = FeaturesFrame(x_train)
test = (
    test
    .with_columns(pl.col('date').str.to_date("%Y-%m-%d"))
    .with_columns(
        pl.col('date').dt.weekday().alias('weekday'),
        # pl.col('date').dt.year().alias('year'), #! useless: only 2023
        pl.col('date').dt.month().alias('month'),
        pl.col('date').dt.day().alias('day')
    )
)
dict_string = test.select(cs.string()).to_dict(as_series=False)
test = (   
    test
    .with_columns(
        encoded_train = LabelEncoder().fit_transform(dict_string['train']),
        encoded_gare = LabelEncoder().fit_transform(dict_string['gare'])
    )
    .drop(cs.string(), cs.date())

)

In [3]:
test

arret,p2q0,p3q0,p4q0,p0q2,p0q3,p0q4,weekday,month,day,encoded_train,encoded_gare
i64,f64,f64,f64,f64,f64,f64,i8,i8,i8,i64,i64
8,0.0,0.0,1.0,-3.0,-1.0,-2.0,1,4,3,24311,34
9,0.0,0.0,0.0,1.0,0.0,1.0,1,4,3,24311,26
10,-1.0,0.0,0.0,-1.0,0.0,0.0,1,4,3,24311,14
11,-1.0,-1.0,0.0,2.0,-2.0,0.0,1,4,3,24311,68
12,-1.0,-1.0,-1.0,-1.0,3.0,2.0,1,4,3,24311,43
…,…,…,…,…,…,…,…,…,…,…,…
16,0.0,1.0,0.0,0.0,-2.0,0.0,2,10,3,1029,68
17,0.0,0.0,1.0,0.0,-2.0,0.0,2,10,3,1029,43
18,1.0,0.0,0.0,0.0,-3.0,0.0,2,10,3,1029,81


In [4]:
StandardScaler().fit(test.to_numpy()).transform(test.to_numpy()).transpose()

array([[-1.46526568, -1.3231836 , -1.18110151, ..., -0.04444485,
         0.09763723,  0.23971931],
       [ 0.0860831 ,  0.0860831 , -0.42864226, ...,  0.60080847,
         0.60080847,  0.60080847],
       [ 0.08531018,  0.08531018,  0.08531018, ...,  0.08531018,
         0.6033934 ,  0.6033934 ],
       ...,
       [-1.55791028, -1.55791028, -1.55791028, ..., -1.55791028,
        -1.55791028, -1.55791028],
       [ 1.09222656,  1.09222656,  1.09222656, ..., -1.61397382,
        -1.61397382, -1.61397382],
       [-0.29245576, -0.61992056, -1.11111777, ...,  1.63139996,
         0.93553725, -0.21058956]])

In [5]:
test = pl.DataFrame(
    data = dict(zip(test.columns, StandardScaler().fit(test.to_numpy()).transform(test.to_numpy()).transpose() )) #! scaler fit que sur train 
)

In [ ]:
test = FeaturesFrame(test).add_feature_interactions().add_feature_square().scale_standard(set="train")[1]

arret,p2q0,p3q0,p4q0,p0q2,p0q3,p0q4,weekday,month,day,encoded_train,encoded_gare,arret_p2q0_multiplied,arret_p3q0_multiplied,arret_p4q0_multiplied,arret_p0q2_multiplied,arret_p0q3_multiplied,arret_p0q4_multiplied,arret_weekday_multiplied,arret_month_multiplied,arret_day_multiplied,arret_encoded_train_multiplied,arret_encoded_gare_multiplied,p2q0_p3q0_multiplied,p2q0_p4q0_multiplied,p2q0_p0q2_multiplied,p2q0_p0q3_multiplied,p2q0_p0q4_multiplied,p2q0_weekday_multiplied,p2q0_month_multiplied,p2q0_day_multiplied,p2q0_encoded_train_multiplied,p2q0_encoded_gare_multiplied,p3q0_p4q0_multiplied,p3q0_p0q2_multiplied,p3q0_p0q3_multiplied,p3q0_p0q4_multiplied,…,p3q0_encoded_gare_multiplied_square,p4q0_p0q2_multiplied_square,p4q0_p0q3_multiplied_square,p4q0_p0q4_multiplied_square,p4q0_weekday_multiplied_square,p4q0_month_multiplied_square,p4q0_day_multiplied_square,p4q0_encoded_train_multiplied_square,p4q0_encoded_gare_multiplied_square,p0q2_p0q3_multiplied_square,p0q2_p0q4_multiplied_square,p0q2_weekday_multiplied_square,p0q2_month_multiplied_square,p0q2_day_multiplied_square,p0q2_encoded_train_multiplied_square,p0q2_encoded_gare_multiplied_square,p0q3_p0q4_multiplied_square,p0q3_weekday_multiplied_square,p0q3_month_multiplied_square,p0q3_day_multiplied_square,p0q3_encoded_train_multiplied_square,p0q3_encoded_gare_multiplied_square,p0q4_weekday_multiplied_square,p0q4_month_multiplied_square,p0q4_day_multiplied_square,p0q4_encoded_train_multiplied_square,p0q4_encoded_gare_multiplied_square,weekday_month_multiplied_square,weekday_day_multiplied_square,weekday_encoded_train_multiplied_square,weekday_encoded_gare_multiplied_square,month_day_multiplied_square,month_encoded_train_multiplied_square,month_encoded_gare_multiplied_square,day_encoded_train_multiplied_square,day_encoded_gare_multiplied_square,encoded_train_encoded_gare_multiplied_square
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-1.465266,0.086083,0.08531,0.620097,-1.467297,-0.30091,-0.352369,-1.441567,-1.216083,-1.55791,1.092227,-0.292456,-0.150739,-0.138783,-0.979862,2.173955,0.585502,0.663018,2.117761,1.780098,2.273517,-1.599571,0.442847,-0.017766,0.052483,-0.014439,-0.006982,-0.010063,-0.128835,-0.0856,-0.154948,0.100499,-0.009989,-0.015745,-0.019814,-0.010132,-0.01471,…,-0.017596,-0.002547,-0.003511,-0.002774,-0.005701,-0.008929,0.012344,-0.014383,-0.017184,-0.002953,-0.003799,0.148665,0.039701,0.268262,0.035892,-0.015417,-0.002919,-0.024908,-0.015028,-0.002405,-0.006651,-0.002528,-0.001731,-0.001998,-0.000836,-0.001937,-0.003202,1.271563,2.5736,1.013612,-0.541802,1.377603,0.511604,-0.562254,1.186005,-0.483858,-0.577292
-1.323184,0.086083,0.08531,0.091992,0.598992,0.06198,0.226497,-1.441567,-1.216083,-1.55791,1.092227,-0.619921,-0.13771,-0.125636,-0.11268,-0.869438,-0.151226,-0.419219,1.912768,1.606858,2.052319,-1.444242,0.848702,-0.017766,0.032114,-0.000506,-0.00332,0.003622,-0.128835,-0.0856,-0.154948,0.100499,-0.038983,-0.017628,-0.004027,-0.007286,-0.003919,…,-0.017557,-0.002557,-0.003513,-0.002781,-0.035694,-0.018359,-0.044869,-0.030252,-0.01766,-0.002959,-0.003845,-0.009169,-0.009784,0.005931,-0.012175,-0.016224,-0.002921,-0.040964,-0.019801,-0.003823,-0.008081,-0.002542,-0.002215,-0.003222,-0.002939,-0.002245,-0.003174,1.271563,2.5736,1.013612,-0.132041,1.377603,0.511604,-0.280077,1.186005,-0.04136,-0.34825
-1.181102,-0.428642,0.08531,0.091992,-0.434153,0.06198,0.033542,-1.441567,-1.216083,-1.55791,1.092227,-1.111118,0.522926,-0.112488,-0.098276,0.480649,-0.138819,-0.074286,1.707776,1.433618,1.83112,-1.288913,1.358501,-0.019594,0.010899,0.010032,-0.00706,-0.005681,0.632158,0.507262,0.770082,-0.500854,0.50578,-0.017628,-0.011921,-0.007286,-0.007516,…,-0.017447,-0.002557,-0.003513,-0.002781,-0.035694,-0.018359,-0.044869,-0